In [1]:
#from gensim.models import KeyedVectors

from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


In [101]:
import pickle
data = pickle.load(open("./Dataset/LDC2015E29/data.p","rb"))
labels = pickle.load(open("./Dataset/LDC2015E29/labels.p","rb"))

In [102]:
for doc in data:
    for i in range(len(doc)):
        doc[i] = doc[i].lower()

In [103]:
x = data
y = [[item[2] for item in label_docs] for label_docs in labels]

In [104]:
i = 69
for a,b in zip(x[i],y[i]):
    if b != 'NONE':
        print(a,"-->",b)

death --> execute
death --> execute
prosecuted --> trialhearing
sentenced --> sentence
death --> execute
death --> execute
charged --> chargeindict
said --> contact
trial --> trialhearing
arrested --> arrestjail
charged --> chargeindict
trial --> trialhearing


In [105]:
counter = collections.Counter()
for doc in x:
    for word in doc:
        counter[word]+=1
#6113 words

In [106]:
label2index = {}
for labels in y:
    for lab in labels:
        if lab not in label2index:
            label2index[lab] = len(label2index)
print(label2index)
for label_of_doc in y:
    for i in range(len(label_of_doc)):
        label_of_doc[i] = label2index[label_of_doc[i]]

{'NONE': 0, 'transferownership': 1, 'arrestjail': 2, 'releaseparole': 3, 'attack': 4, 'transportperson': 5, 'transportartifact': 6, 'trialhearing': 7, 'extradite': 8, 'broadcast': 9, 'convict': 10, 'transaction': 11, 'contact': 12, 'acquit': 13, 'die': 14, 'injure': 15, 'divorce': 16, 'chargeindict': 17, 'endposition': 18, 'pardon': 19, 'correspondence': 20, 'sentence': 21, 'appeal': 22, 'demonstrate': 23, 'meet': 24, 'transfermoney': 25, 'elect': 26, 'startposition': 27, 'startorg': 28, 'sue': 29, 'declarebankruptcy': 30, 'marry': 31, 'nominate': 32, 'fine': 33, 'endorg': 34, 'execute': 35, 'beborn': 36, 'mergeorg': 37, 'artifact': 38}


In [107]:
print(len(label2index))

39


In [108]:
WINDOW_SIZE = 5
VOCAB_SIZE = 7000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 20
WORD2VEC_MODEL = "./Dataset/Glove-vectorsr/glove.6B.300d.txt"

In [109]:
word2emb = {}
fglove = open(WORD2VEC_MODEL,"r")
for line in fglove:
    cols = line.strip().split()
    word = cols[0]
    embedding = np.array(cols[1:],dtype="float32")
    word2emb[word]=embedding
fglove.close()

In [110]:
word2index = collections.defaultdict(int)
for wid,word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid+1
vocab_size = len(word2index)+1
index2word = {v : k for k,v in word2index.items()}


In [111]:
embeddings = np.zeros((vocab_size,EMBED_SIZE))
for word,index in word2index.items():
    try:
        embeddings[index,:] = word2emb[word]
    except KeyError:
        pass

In [112]:
whole_data = []
whole_labels = []
maxlen = 0
for doc,label in zip(x,y):
    for i in range(len(doc)):
        lower = max(0,i-WINDOW_SIZE)
        upper = i + WINDOW_SIZE
        text = doc[lower:upper]
        maxlen = max(maxlen,len(text))
        for j in range(len(text)):
            text[j] = word2index[text[j]]
        whole_data.append(text)
        whole_labels.append(label[i])
        
        
        
        

In [113]:
whole_data[0:20]

[[2841, 1320, 4, 70, 676],
 [2841, 1320, 4, 70, 676, 6],
 [2841, 1320, 4, 70, 676, 6, 2842],
 [2841, 1320, 4, 70, 676, 6, 2842, 1034],
 [2841, 1320, 4, 70, 676, 6, 2842, 1034, 383],
 [2841, 1320, 4, 70, 676, 6, 2842, 1034, 383, 2843],
 [1320, 4, 70, 676, 6, 2842, 1034, 383, 2843, 225],
 [4, 70, 676, 6, 2842, 1034, 383, 2843, 225, 677],
 [70, 676, 6, 2842, 1034, 383, 2843, 225, 677, 6],
 [676, 6, 2842, 1034, 383, 2843, 225, 677, 6, 70],
 [6, 2842, 1034, 383, 2843, 225, 677, 6, 70, 436],
 [2842, 1034, 383, 2843, 225, 677, 6, 70, 436, 486],
 [1034, 383, 2843, 225, 677, 6, 70, 436, 486, 2844],
 [383, 2843, 225, 677, 6, 70, 436, 486, 2844, 2],
 [2843, 225, 677, 6, 70, 436, 486, 2844, 2, 1321],
 [225, 677, 6, 70, 436, 486, 2844, 2, 1321, 1322],
 [677, 6, 70, 436, 486, 2844, 2, 1321, 1322, 5],
 [6, 70, 436, 486, 2844, 2, 1321, 1322, 5, 2845],
 [70, 436, 486, 2844, 2, 1321, 1322, 5, 2845, 84],
 [436, 486, 2844, 2, 1321, 1322, 5, 2845, 84, 1035]]

In [114]:
whole_labels[0:20]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [115]:
maxlen

10

In [116]:
X = pad_sequences(whole_data,maxlen=maxlen)
Y = np_utils.to_categorical(whole_labels)

In [117]:
X[0:10],Y[0:10]

(array([[   0,    0,    0,    0,    0, 2841, 1320,    4,   70,  676],
        [   0,    0,    0,    0, 2841, 1320,    4,   70,  676,    6],
        [   0,    0,    0, 2841, 1320,    4,   70,  676,    6, 2842],
        [   0,    0, 2841, 1320,    4,   70,  676,    6, 2842, 1034],
        [   0, 2841, 1320,    4,   70,  676,    6, 2842, 1034,  383],
        [2841, 1320,    4,   70,  676,    6, 2842, 1034,  383, 2843],
        [1320,    4,   70,  676,    6, 2842, 1034,  383, 2843,  225],
        [   4,   70,  676,    6, 2842, 1034,  383, 2843,  225,  677],
        [  70,  676,    6, 2842, 1034,  383, 2843,  225,  677,    6],
        [ 676,    6, 2842, 1034,  383, 2843,  225,  677,    6,   70]], dtype=int32),
 array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [198]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size = 0.1, random_state=42)

In [118]:
data = {}
for x,y in zip(X,Y):
    index = np.argmax(y)
    if index not in data:
        data[index] = [(x,y)]
    else:
        data[index] += [(x,y)]
    

In [119]:
src_model = Sequential()
src_model.add(Embedding(vocab_size, EMBED_SIZE, input_length=maxlen,trainable=False,weights=[embeddings]))
dst_model = Sequential()
dst_model.add(Embedding(maxlen, 50, input_length=maxlen))

In [120]:
Xtrain = []
Ytrain = []
Xtest = []
Ytest = []

for index in data:
    curr = data[index]
    leng = len(curr)
#    trainx = []
#    trainy = []
#    testx = []
#    testy = []
    for i in range(int(0.8*leng)):
        Xtrain.append(curr[i][0])
        Ytrain.append(curr[i][1])
    for i in range(int(0.8*leng),leng):
        Xtest.append(curr[i][0])
        Ytest.append(curr[i][1])
        
        
        

In [121]:
import random
combined = list(zip(Xtrain, Ytrain))
random.shuffle(combined)

Xtrain[:], Ytrain[:] = zip(*combined)

In [122]:
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)

In [123]:
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)

In [124]:
model = Sequential()
#model.add(Embedding(vocab_size,EMBED_SIZE,input_length=maxlen,weights=[embeddings],trainable=False))
from keras.layers import Merge
model.add(Merge([src_model, dst_model], mode='concat',concat_axis=-1))
#model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=1200,kernel_size=NUM_WORDS,activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(39,activation="softmax"))

/home/siddharth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.


In [64]:
x1 = list(range(0,10))
x1 = [x1]*(len(Xtrain)+len(Xtest))
x1 = np.array(x1)

In [65]:
x1

array([[0, 1, 2, ..., 7, 8, 9],
       [0, 1, 2, ..., 7, 8, 9],
       [0, 1, 2, ..., 7, 8, 9],
       ..., 
       [0, 1, 2, ..., 7, 8, 9],
       [0, 1, 2, ..., 7, 8, 9],
       [0, 1, 2, ..., 7, 8, 9]])

In [125]:
from keras import metrics
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
history = model.fit(x=[Xtrain,x1[:len(Xtrain)]],y=Ytrain,batch_size=BATCH_SIZE,epochs=10,shuffle=True,validation_data=([Xtest,x1[len(Xtrain):]],Ytest))
#a = model.predict(Xtest)

Train on 27763 samples, validate on 6957 samples
Epoch 1/10
27763/27763 [==============================] - 33s - loss: 0.3306 - acc: 0.9461 - val_loss: 0.2374 - val_acc: 0.9503
Epoch 2/10
27763/27763 [==============================] - 35s - loss: 0.1498 - acc: 0.9602 - val_loss: 0.2119 - val_acc: 0.9556
Epoch 3/10
27763/27763 [==============================] - 34s - loss: 0.0949 - acc: 0.9729 - val_loss: 0.2074 - val_acc: 0.9553
Epoch 4/10
27763/27763 [==============================] - 34s - loss: 0.0661 - acc: 0.9786 - val_loss: 0.2103 - val_acc: 0.9577
Epoch 5/10
27763/27763 [==============================] - 35s - loss: 0.0561 - acc: 0.9822 - val_loss: 0.2323 - val_acc: 0.9585
Epoch 6/10
27763/27763 [==============================] - 36s - loss: 0.0423 - acc: 0.9863 - val_loss: 0.2448 - val_acc: 0.9582
Epoch 7/10
27763/27763 [==============================] - 34s - loss: 0.0387 - acc: 0.9890 - val_loss: 0.2797 - val_acc: 0.9566
Epoch 8/10
27763/27763 [==============================]

In [126]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 10, 350)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 8, 1200)           1261200   
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 1200)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 39)                46839     
Total params: 3,142,739
Trainable params: 1,308,539
Non-trainable params: 1,834,200
_________________________________________________________________


In [127]:
a = model.predict(x = [Xtest,x1[len(Xtrain):]])
#a = np.argmax(a,axis=1)
a[0]

array([  9.99991894e-01,   2.05105923e-08,   3.05658219e-08,
         2.32493380e-09,   5.17462080e-11,   2.02266790e-08,
         1.24838362e-10,   2.39948550e-09,   3.04735848e-10,
         6.96858524e-06,   1.52979573e-11,   1.17956755e-10,
         5.20895355e-07,   5.16014922e-12,   4.99033731e-11,
         5.33848844e-13,   2.75524026e-09,   1.71357858e-10,
         1.63002785e-08,   7.36253836e-10,   1.28240117e-07,
         7.44221698e-12,   6.49578169e-10,   9.12887613e-11,
         4.80671414e-09,   3.68737290e-07,   4.97078934e-10,
         3.42541107e-08,   1.21160526e-10,   5.32976441e-11,
         1.59941962e-10,   3.57339193e-11,   2.28267072e-10,
         2.61486588e-09,   3.83009206e-12,   1.18939784e-13,
         1.94351743e-10,   4.08784617e-10,   4.73061521e-12], dtype=float32)

In [128]:
np.argmax(a[100])
print(a[100])

[  9.99999642e-01   1.58635960e-09   7.99174338e-11   3.68040251e-11
   1.62056521e-10   4.07214089e-11   1.52953428e-10   8.23392465e-11
   3.43105888e-13   2.32682396e-07   6.26998930e-12   1.45222750e-10
   5.39068630e-08   1.37853691e-13   4.76008393e-13   3.30081286e-13
   6.05558355e-12   7.50505387e-12   3.10925140e-11   7.04408146e-12
   6.59372390e-10   2.57144323e-11   1.17544030e-09   3.17993930e-11
   2.15498938e-11   5.12833118e-08   3.40447844e-11   2.08378606e-10
   7.37445001e-12   3.52073128e-11   4.50982272e-11   1.07899703e-12
   1.48180756e-11   4.17615803e-11   1.70055891e-13   4.17687282e-12
   1.73402091e-12   4.87486483e-12   1.07534814e-12]


In [129]:
ytest = np.argmax(Ytest,axis=1)

In [130]:
len(x1[len(Xtrain):]) == len(Xtest)
a = np.argmax(a,axis=1)

In [136]:

table = np.zeros((39,3),dtype="int32")
#first -> total prediced
#second -> total actual
#third -> correct
for c,d in zip(a,ytest):
    table[c][0]+=1
    table[d][1]+=1
    if(c==d):
        table[c][2]+=1
for i ,element in enumerate(table):
    p = element[2]*1.0/(element[0]+1e-5)
    r = element[2]*1.0/(element[1]+1e-5)
    f = 2*p*r/(p+r+1e-5)
    print(index2label[i],"-->",f)
for i,j in enumerate(table):
    print(index2label[i],"----->",j)

NONE --> 0.981283666583
transferownership --> 0.28570993203
arrestjail --> 0.555550262391
releaseparole --> 0.842099390612
attack --> 0.606055427038
transportperson --> 0.474571123293
transportartifact --> 0.0
trialhearing --> 0.521734139926
extradite --> 0.0
broadcast --> 0.24242042246
convict --> 0.761899047652
transaction --> 0.333327777837
contact --> 0.0689609515439
acquit --> 0.0
die --> 0.551719096357
injure --> 0.444439012401
divorce --> 0.285708571514
chargeindict --> 0.545449421527
endposition --> 0.624994785194
pardon --> 0.933327111139
correspondence --> 0.222218271653
sentence --> 0.8965461118
appeal --> 0.666659444489
demonstrate --> 0.428566734731
meet --> 0.315785263204
transfermoney --> 0.540535339709
elect --> 0.46153278112
startposition --> 0.285708877639
startorg --> 0.0
sue --> 0.0
declarebankruptcy --> 0.0
marry --> 0.0
nominate --> 0.199994600126
fine --> 0.749993125038
endorg --> 0.66665777783
execute --> 0.0
beborn --> 0.0
mergeorg --> 0.499993750028
artifact -

In [74]:
#import keras.backend as K
#def f_score(y_true,y_pred):


In [167]:
ytest = np.argmax(testy,axis=1)

In [88]:
ytest

array([ 0,  0,  0, ...,  0,  0, 13])

In [95]:
table

array([[10105,  9857,  9597],
       [   12,    23,     1],
       [    2,    24,     0],
       [    5,    21,     2],
       [   14,    44,     5],
       [   30,    43,     2],
       [    2,     9,     0],
       [    2,    29,     0],
       [    0,     3,     0],
       [    8,    32,     0],
       [   27,    15,     1],
       [    3,     5,     0],
       [    2,    30,     0],
       [    0,     5,     0],
       [   23,    47,     5],
       [    9,    11,     0],
       [    0,     5,     0],
       [   29,    14,     1],
       [   17,    26,     1],
       [    9,     9,     2],
       [    0,     9,     0],
       [   47,    21,     8],
       [    0,     2,     0],
       [   12,    17,     1],
       [    9,    22,     0],
       [   28,    31,     2],
       [    8,     9,     0],
       [    2,    13,     0],
       [    0,     5,     0],
       [    0,     4,     0],
       [    0,     0,     0],
       [    1,     6,     0],
       [    0,     6,     0],
       [  

In [203]:
?? Embedding

In [26]:
?? Dropout

In [70]:
?? Merge


In [44]:
?? Embedding

In [54]:
?? model.fit

In [99]:
label2index

{'NONE': 0,
 'business': 8,
 'conflict': 3,
 'contact': 5,
 'justice': 2,
 'life': 6,
 'manufacture': 9,
 'movement': 4,
 'personnel': 7,
 'transaction': 1}

In [133]:
index2label = {v:k for k,v in label2index.items()}

In [134]:
index2label

{0: 'NONE',
 1: 'transferownership',
 2: 'arrestjail',
 3: 'releaseparole',
 4: 'attack',
 5: 'transportperson',
 6: 'transportartifact',
 7: 'trialhearing',
 8: 'extradite',
 9: 'broadcast',
 10: 'convict',
 11: 'transaction',
 12: 'contact',
 13: 'acquit',
 14: 'die',
 15: 'injure',
 16: 'divorce',
 17: 'chargeindict',
 18: 'endposition',
 19: 'pardon',
 20: 'correspondence',
 21: 'sentence',
 22: 'appeal',
 23: 'demonstrate',
 24: 'meet',
 25: 'transfermoney',
 26: 'elect',
 27: 'startposition',
 28: 'startorg',
 29: 'sue',
 30: 'declarebankruptcy',
 31: 'marry',
 32: 'nominate',
 33: 'fine',
 34: 'endorg',
 35: 'execute',
 36: 'beborn',
 37: 'mergeorg',
 38: 'artifact'}